# Intrusion Detection Computer Vision System

## Task 1 (Mandatory):
### Graphical Output
For each frame of the input video the system needs to show found blobs (either by coloring them on a black background or by showing the countours over the original video)
### Text Output
For each frame print the number of found objects, the value associated with each feature of the blob and its classification into person or other.

## Task 2 (Optional)
Develop an algorithm to distinguish between true objects and the removal of a previously present one.

## Video Characteristics
- 12 fps
- ~41s
- 320x240 pixels
- 8 bit/pixel (256 gray levels)

In [ ]:
# Imports

import cv2
import numpy as np
from matplotlib import pyplot as plt
from IPython import display

In [ ]:
# Global variables

input_video_path = "rilevamento-intrusioni-video.avi"
output_video_path = "test.avi"

In [ ]:
# Video Helper Functions

def play_video(video_path):
    '''
        Plays the video found in video_path frame by frame
    '''
    cap = cv2.VideoCapture(video_path)
    
    try:
        while True:
            # Capture frame-by-frame
            ret, frame = cap.read()
            if not ret or frame is None:
                cap.release()
                print("Released Video Resource")
                break
            
            # Display frame
            plt.axis('off')
            plt.imshow(frame)
            plt.show()
            
            # Clear cell output when new frame is available
            display.clear_output(wait=True)
    except KeyboardInterrupt:
        cap.release()
        print("Released Video Resource")    

def edit_video(input_video_path, output_video_path, frame_transformation):
    '''
        Applies frame_transformation function to each frame taken from input_video_path and saves the result in
        output_video_path
    '''
    cap = cv2.VideoCapture(input_video_path)

    # Getting original video params
    w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)

    # Define the codec and create VideoWriter object
    fourcc = cv2.VideoWriter_fourcc(*'DIVX')
    out = cv2.VideoWriter(output_video_path, fourcc, fps, (w,  h))

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret or frame is None:
            print("Can't receive frame (stream end?). Exiting ...")
            break
        frame = frame_transformation(frame)
        # write the updated frame
        out.write(frame)
    cap.release()
    out.release()
    
def create_output_stream(cap, output_video_path):
    '''
        Saves cap in output_video_path
    '''
    # Getting original video params
    w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)

    # Define the codec and create VideoWriter object
    fourcc = cv2.VideoWriter_fourcc(*'DIVX')
    out = cv2.VideoWriter(output_video_path, fourcc, fps, (w,  h))

    return out

In [ ]:
# Frame Editing Functions

def binarize_mask(mask, threshold=None):
    ''' This method takes the mask and binarize it. The return is a mask of float64 with values 0 or 255
    '''
    res = np.zeros(mask.shape)
    if mask.dtype == bool:
        res[mask] = 255
    else:
        res[mask < threshold] = 255
    return res

def gaussian_filter(frame, sigma=1.5, k_size=None):
    '''
        Higher sigmas should correspond to larger kernels. usually big as 
        Rule of thumb for a good kernel size given sigma
    '''
    if k_size is None:
        k_size = int(np.ceil((3*sigma))*2 + 1)
 
    return cv2.GaussianBlur(frame, (k_size,k_size) , sigma)

def preprocessing(frame, parameters):
    if parameters["preprocessing"] is not None:
        function, params = parameters["preprocessing"]
        return function(frame, *params)
    return frame

In [ ]:
# Distance Functions

def manhattan_distance(img1, img2):
    ''' returns a matrix of floats64 that represents the manhattan distance between the two images
    '''
    return np.sum(np.abs(img2 - img1), axis=-1)

def euclidean_distance(img1, img2):
    ''' returns a matrix of floats64 that represents the euclidean distance between the two images
    '''
    return np.sqrt(np.sum((img2 - img1) ** 2, axis=-1))

def maximum_distance(img1, img2):
    '''returns a matrix of floats64 that represents the Chebyshev distance between the two images
    '''
    return np.max(img2 - img1, axis=-1)

In [ ]:
# Change Detection Functions

def two_frame_difference(prev_frame, curr_frame, d_func, threshold):
    ''' This method applies the distance function (d_func) on the two frames returning a matrix of floats64
    '''
    return d_func(curr_frame, prev_frame) > threshold

def three_frame_difference(prev_frame, curr_frame, next_frame, d_func, threshold):
    '''Computes the TFD between the first and second frame and the TFD between the second and the third frame, after that it computes and returns as matrix of Booleans the logical and between the two TFD.
    '''
    diff1 = two_frame_difference(prev_frame, curr_frame, d_func, threshold)
    diff2 = two_frame_difference(curr_frame, next_frame, d_func, threshold)
    and_mask = np.prod([diff1, diff2],axis=0, dtype=bool)
    return and_mask

def background_subtraction(frame, background, d_func, threshold):
    '''Computes the Background subtraction (distance(frame,background)) and returns a matrix of boolean TODO: VERIFICARE CHE RITORNI EFFETTIVAMENTE UNA MATRICE DI BOOLEANI
    '''
    frame = frame.astype(float)
    mask = d_func(frame, background) > threshold
    return mask

def background_set_initialization(input_video_path, parameter_set):
    ''' TODO: Initializes the parameters set for the background subtraction phase,
        Returns a list of sets containing the parameters of that run.
    '''
    bs = []
    for params in compute_parameters(parameter_set):
        cap = cv2.VideoCapture(input_video_path)
        bs.append({
            "image": background_initialization(cap, params["interpolation"], params["frames"]),
            "name": "{}_{}".format(params["frames"], params["interpolation"].__name__)
        })
    return bs
    

def background_initialization(cap, interpolation, n=100):
    '''Creates the background that will be used in the method backgorund_subtraction(...) stacking the first n frames and calling on them the <interpolation> method (np.mean or np.median) 
        Returning a matrix of float64
    '''
    # Loading Video
    bg = []
    idx = 0
    # Initialize the background image
    while(cap.isOpened() and idx < n):
        ret, frame = cap.read()
        if ret and not frame is None:
            frame = frame.astype(float)
            # Getting all first n images
            bg.append(frame)
            idx += 1
        else:
            break
    cap.release()

    bg_interpolated = np.stack(bg, axis=0)
    return interpolation(bg_interpolated, axis=0)

def initialize_subtractor():
    '''creates and returns a MOG2 background subtractor
    '''
    fgbg = cv2.createBackgroundSubtractorMOG2(history=20, varThreshold=25)
    fgbg.setDetectShadows(False)
    return fgbg

def background_subtraction_mog2(frame, fgbg):
    '''Applies the MOG2 background subtractor to the frame and returns the foreground mask as 8-bit binary image
    '''
    return fgbg.apply(frame)

def change_detection(frame, parameters):
    ''' Calls the background subtraction method using {<image> <background image> <Distance function> <Threshold> ...<>} as inputs
        Returns a matrix of boolean (see: backgorund_subtraction(...))
    '''
    #mask = background_subtraction_mog2(frame, parameters["subtractor"])
    #mask = background_subtraction(frame, parameters["background"]["image"], parameters["distance"], parameters["threshold"])
    back = blind_background_subtraction(frame,0.02)
    mask = background_subtraction(frame, back, parameters["distance"], parameters["threshold"])
    return mask


#Blind Background Subtraction

#Per ogni step t devo calcolaare il background utilizzando: alfa x frame + (1-alfa)x background precedente 
previous_background = None
def blind_background_subtraction(frame, alfa):
    
    global previous_background
    new_bg = previous_background*(1-alfa)
    new_bg = new_bg + frame*alfa
    previous_background = new_bg
    
    
    return new_bg
    

In [ ]:
# Binary Morphology Functions

def bm_test(mask):
    '''Applies the binary morphology on the mask and returns it as a matrix of Boolean
    '''
    mask = mask.astype(np.uint8)
    kernel1 = np.ones((3,3), np.uint8)
    kernel2 = np.ones((30,30), np.uint8)

    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel1)
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel2)
    return mask.astype(bool)

def binary_morphology(mask, parameters):
    '''Calls the the method contained in parameters['morphology'] and returns the resulting mask after applying the morphology as a matrix of Boolean TODO: Assicurarsi che la morphology returna sempre una matrice
        di booleani, se non lo fa, accertarsi di cambiare questa descrizione
    '''
    if parameters['morphology'] is None:
        return mask
    return parameters['morphology'](mask)

In [ ]:
# Blob Analysis Functions
import csv

def initialize_blob_detector():
    '''Initializes the parameters of the blob detector and returns the detector
    '''
    params = cv2.SimpleBlobDetector_Params()

    # Change thresholds
    #params.minThreshold = 10
    #params.maxThreshold = 200

    # Filter by Area.
    params.filterByArea = False
    #params.minArea = 1500

    # Filter by Circularity
    params.filterByCircularity = False
    #params.minCircularity = 0.1

    # Filter by Convexity
    params.filterByConvexity = False
    #params.minConvexity = 0.87

    # Filter by Inertia
    params.filterByInertia = False
    #params.minInertiaRatio = 0.01
    
    params.filterByColor = True
    params.blobColor = 255
    
    detector = cv2.SimpleBlobDetector_create(params)
    return detector

def blob_detection(mask, frame, parameters):
    '''Detects the blobs and creates them. Returns the blobs (as dictionaries) and the respective frames with the contour drawn on it
    ''' 
    #keypoints = parameters['blob_detector'].detect(mask)
    #frame_parsed = cv2.drawKeypoints(frame, keypoints, np.array([]), (0,0,255), cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
    ret, thresh = cv2.threshold(mask[:,:,0], 0, 255, 0)
    im2, contours, hierarchy = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    #print(len(contours[0]))
    colored_frame = frame.copy()
    cv2.drawContours(colored_frame, contours, -1, (0,255,0), 3)
    
    blobs = []
    for contour in contours:
        blobs.append({
            'label': 1,
            'area': 10,
            'perimeter': 20,
            'classification': 'person'
            
        })
    return blobs, colored_frame

def append_text_output(frame_index, blobs, csv_writer):
    '''Writes on the csv document the infos of the blob.
    '''
    csv_writer.writerow([frame_index, len(blobs)])
    for blob in blobs:
        csv_writer.writerow([blob['label'], blob['area'], blob['perimeter'], blob['classification']])

In [ ]:
import itertools

def prepare_output(mask):
    ''' This method binarizes the mask (float64) and then returns the mask as np.uint8 matrix with 3 dimensions
    '''
    mask = binarize_mask(mask,120)
    return np.uint8(np.tile(mask[:,:,np.newaxis], 3))
    

def compute_parameters(param_set):
    ''' Computes the parameters that needs to be used for the iteration of image elaboration. Returns a cartesian product of the methods and parameters that needs to be used. 
    '''
    return (dict(zip(param_set, x)) for x in itertools.product(*param_set.values()))

def generate_filename(directory, parameters, extension):
    '''Returns a string for the filename containing the information of the parameters used in that run
    '''
    preprocessing_name = "none"
    if parameters["preprocessing"] is not None:
        function, params = parameters["preprocessing"]
        preprocessing_name = "_{}_{}".format(function.__name__, params)
       
    morphology_name = "none"
    if parameters["morphology"] is not None:
        morphology_name = parameters["morphology"].__name__
            
    background_name = "none"
    if parameters["background"] is not None:
        background_name = parameters["background"]["name"]
    
    return "{}tuning_{}_{}_{}_{}_{}.{}".format(
        directory, 
        background_name,
        parameters["threshold"], 
        parameters["distance"].__name__,
        preprocessing_name,
        morphology_name,
        extension
    )


In [ ]:
# Parameter Tuning

output_dir = "output/"

background_parameters_set = {
    "frames": [ 130],
    "interpolation": [ np.median]
}

parameters_set = {
    "threshold": [35, 45],
    "distance": [euclidean_distance],
    "preprocessing": [None],
    "morphology": [bm_test],
    "background": background_set_initialization(input_video_path, background_parameters_set),
    "subtractor": [initialize_subtractor()],
    "blob_detector": [initialize_blob_detector()]
}

                    
def compute_intrusion_detection(input_video_path, output_video_path, output_text_path, parameters):
    ''' Runs the elaboration of the video, and writes on the csv document the informations of the blobs found and creates a video in the "output_video_path" that shows the effects of the elaborations on the video.
    '''
    cap = cv2.VideoCapture(input_video_path)
    out = create_output_stream(cap, output_video_path)
    try:
        #csv_file = open(output_text_path, mode='w')
        #csv_writer = csv.writer(csv_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
        global previous_background
        previous_background = parameters["background"]["image"]
      
        idx = 0
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret or frame is None:
                print("Computation finished! Video saved in {}".format(output_video_path))
                break

            preprocessed_frame = preprocessing(frame, parameters)
            mask_raw = change_detection(frame, parameters)
            #mask_refined = binary_morphology(mask_raw, parameters)
            mask_output = prepare_output(mask_raw)
            #blobs, colored_frame = blob_detection(mask_output, frame, parameters)
            #append_text_output(idx, blobs, csv_writer)
            out.write(mask_output)
            idx += 1
    finally:
        #csv_file.close()
        pass
        

    out.release()
    cap.release()

In [ ]:
# Execution
''' Calls the elaboration of the video with every result of the cartesian product of the parameters.
'''
for parameters in compute_parameters(parameters_set):
    compute_intrusion_detection(
        input_video_path, 
        generate_filename(output_dir, parameters, 'avi'), 
        generate_filename(output_dir, parameters, 'csv'), 
        parameters)
print("Finished!")